## Homework

Исходные данные - Yandex Cup 2022 RecSys:
- Описание соревнования - https://contest.yandex.ru/yacup/contest/41618/problems/
- Данные - https://disk.yandex.ru/d/SI1aAooPn9i8TA
- Описание данных - в архиве likes_data.zip три файла:
  - train - обучающий набор данных. Каждая строка - последовательность id треков, которые лайкнул один пользователь. Гарантируется, что лайки даны в той последовательности, в которой их ставил пользователь.
  - test - набор тестовых данных. Имеет точно такой же формат, но в каждой строке не хватает последнего лайка, который надо предсказать.
  - track_artists.csv - информация о исполнителях треков. Гарантируется, что у каждого трека есть ровно один исполнитель. Для треков, у которых фактически несколько исполнителей, мы оставили того, который считается основным исполнителем трека.
- Описание сабмита - в качестве решения необходимо отправить файл, в котором для каждого пользователя из test в отдельной строке будет не более 100 треков, разделенных пробелом. Гарантируется, что у каждого пользователя будет только 1 лайк в тесте
- Метрика - MRR@100

Промежуточная задача - преобразовать данные в pandas.DataFrame вида {user, item, order}, где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

**Итоговая задача** - построить схему валидации для данного соревнования с учетом особенностей сорвенования
- Между `train` и `test` не должно быть общих пользователей
- Количество фолдов задается через параметр класса `n_folds`
- В `test` должно быть не более `p` последних треков (параметр класса `p`)

**I. Промежуточная задача**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Вар. 2 Пробуем сделать более эффективное считывание из файла. Использую pandas, как посоветовал ментор.

In [8]:
ya_data = pd.read_csv('/content/drive/MyDrive/hw2/test', sep='/n', names=['item_id'])
ya_data.head(2)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,item_id
0,454758 382341 240893 280388 362253 436737 2576...
1,294661 374689 182272 102025 70927 86844 280488...


In [9]:
ya_data['user_id'] = ya_data.index
ya_data.head()

,item_id,user_id
0,454758 382341 240893 280388 362253 436737 2576...,0
1,294661 374689 182272 102025 70927 86844 280488...,1
2,15296 384304 138151 474141 182935 262020 21145...,2
3,166224 30240 75272 291190 362083 443361 451254...,3
4,212560 474729 466936 196474 86487 375368 30801...,4


In [10]:
ya_data = ya_data.assign(item_id=ya_data.item_id.str.split(' ')).explode('item_id').reset_index(drop=True)
print(ya_data)

         item_id  user_id
0         454758        0
1         382341        0
2         240893        0
3         280388        0
4         362253        0
...          ...      ...
23262195  448288   289913
23262196    1343   289913
23262197   86420   289913
23262198  186436   289913
23262199    8474   289913

[23262200 rows x 2 columns]


In [18]:
ya_data = ya_data.reindex(columns=['user_id', 'item_id'])

In [24]:
 # где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)
ya_data['order'] = ya_data.groupby('user_id')['item_id'].cumcount()

In [26]:
ya_data.head(60)

,user_id,item_id,order
0,0,454758,0
1,0,382341,1
2,0,240893,2
3,0,280388,3
4,0,362253,4
5,0,436737,5
6,0,257617,6
7,0,92649,7
8,0,366042,8
9,0,102998,9


In [27]:
ya_data.info

<bound method DataFrame.info of           user_id item_id  order
0               0  454758      0
1               0  382341      1
2               0  240893      2
3               0  280388      3
4               0  362253      4
...           ...     ...    ...
23262195   289913  448288     34
23262196   289913    1343     35
23262197   289913   86420     36
23262198   289913  186436     37
23262199   289913    8474     38

[23262200 rows x 3 columns]>

Вар. 1 (чтение файла не через pandas)

In [ ]:
def get_dataset(path_to_data, count = 0):
  likes_data_list = []

  with open(path_to_data, 'r') as f:
    lines = f.readlines()

    for idx, line in enumerate(lines):
      if idx > count:
        break
      items = [int(i) for i in line.split()]
      likes_data = np.empty((len(items), 3), dtype=int)
      likes_data[:, 0] = idx # user_id
      likes_data[:, 1] = items[::-1] # track_id
      likes_data[:, 2] = np.arange(len(items)) # order
      likes_data_list.append(likes_data)
    
  array_data = np.vstack(likes_data_list)

  df = pd.DataFrame(array_data, columns = ['user_id', 'track_id', 'order'])

  return df

In [ ]:
path_to_data = '/content/drive/MyDrive/hw2/test'

example = get_dataset(path_to_data, count = 1000)
example.head() 

,user_id,track_id,order
0,0,471705,0
1,0,219426,1
2,0,101168,2
3,0,361110,3
4,0,22932,4


**II. Итоговая задача**

In [22]:
class UsersKFoldPOut():
    def __init__(self, n_folds, p, random_seed = 23):
      self.n_folds = n_folds
      self.p = p
      self.random_seed = random_seed
    
    def split(self, df):
      df = df.copy()  
      users = df['user_id'].unique()
      # разбивка по фолдам
      users_kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_seed)

      for train_users, test_users in users_kfold.split(users):
        train_mask = df['user_id'].isin(train_users)
        test_mask = df['user_id'].isin(test_users) & (df['order'] < self.p)
        yield train_mask, test_mask

In [ ]:
n_folds = 3
p = 1
cv = UsersKFoldPOut(n_folds = n_folds, p = p)

for i, (train_mask, test_mask) in enumerate(cv.split(example)):
  train = example[train_mask]
  test = example[test_mask]

  # Проверка на отсутствие общих пользователей между train и test
  assert set() == set(train['user_id']).intersection(test['user_id'])
  # Проверка на соответствие условию: "В test должно быть не более p последних треков (параметр класса p)"
  assert test[test['order'] > p].shape[0] == 0

  print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

Fold#0 | Train: 56853, Test: 334
Fold#1 | Train: 56087, Test: 334
Fold#2 | Train: 57452, Test: 333


Работа с датасетом предобработанным с помощью Pandas

In [23]:
n_folds = 3
p = 1
cv = UsersKFoldPOut(n_folds = n_folds, p = p)

for i, (train_mask, test_mask) in enumerate(cv.split(ya_data)):
  train = ya_data[train_mask]
  test = ya_data[test_mask]

  # Проверка на отсутствие общих пользователей между train и test
  assert set() == set(train['user_id']).intersection(test['user_id'])
  # Проверка на соответствие условию: "В test должно быть не более p последних треков (параметр класса p)"
  assert test[test['order'] > p].shape[0] == 0

  print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

Fold#0 | Train: 15516536, Test: 96638
Fold#1 | Train: 15539176, Test: 96638
Fold#2 | Train: 15468688, Test: 96638
